In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Essential_Thrombocythemia/GSE12295'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Class prediction models of thrombocytosis using genetic biomarkers"
!Series_summary	"Using custom spotted oligonucelotide platelet-focused arrays, platelet samples were compared. 48 health controls, 23 reactive thrombocytosis (RT) and 24 essential thrombocythemia (ET) samples were compared. An 11-biomarker gene subset was identified that discriminated among the three cohorts with 86.3% accuracy."
!Series_overall_design	"70 mer oligonucleotides were spotted in quadruplicate and hybridized versus reference RNA in two color method. Spiked control RNAs were also included."
Sample Characteristics Dictionary:
{0: ['Essential thrombocythemia Patient Platelet', 'Reactive Thrombocytosis Patient platelets', 'Normal Patient Platelets']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is available
is_gene_available = True

# Identify rows for Essential_Thrombocythemia, age, and gender
trait_row = 0
age_row = None
gender_row = None

# Define conversion functions
def convert_trait(value):
    trait_map = {
        'Essential thrombocythemia Patient Platelet': 1,
        'Reactive Thrombocytosis Patient platelets': None,
        'Normal Patient Platelets': 0
    }
    key_value = value.strip()
    return trait_map.get(key_value, None)

# Note: No need for convert_age and convert_gender as age_row and gender_row are None

# Save cohort info
save_cohort_info('GSE12295', './preprocessed/Essential_Thrombocythemia/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction if trait_row is not None
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Essential_Thrombocythemia', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Essential_Thrombocythemia/trait_data/GSE12295.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM309072': [1], 'GSM309073': [1], 'GSM309074': [1], 'GSM309075': [1], 'GSM309076': [1], 'GSM309077': [1], 'GSM309078': [1], 'GSM309079': [1], 'GSM309080': [1], 'GSM309081': [1], 'GSM309082': [1], 'GSM309083': [1], 'GSM309084': [1], 'GSM309085': [1], 'GSM309086': [1], 'GSM309087': [1], 'GSM309088': [1], 'GSM309089': [None], 'GSM309090': [1], 'GSM309091': [None], 'GSM309092': [1], 'GSM309093': [1], 'GSM309094': [1], 'GSM309095': [0], 'GSM309096': [0], 'GSM309097': [0], 'GSM309098': [0], 'GSM309099': [0], 'GSM309100': [0], 'GSM309101': [0], 'GSM309102': [0], 'GSM309103': [0], 'GSM309104': [0], 'GSM309105': [0], 'GSM309106': [0], 'GSM309107': [0], 'GSM309108': [0], 'GSM309109': [0], 'GSM309110': [0], 'GSM309111': [0], 'GSM309112': [0], 'GSM309113': [0], 'GSM309114': [0], 'GSM309115': [0], 'GSM309116': [0], 'GSM309117': [0], 'GSM309118': [0], 'GSM309119': [0], 'GSM309120': [0], 'GSM309121': [0], 'GSM309122': [0], 'GSM309123': [0], 'GSM309124': [0], 'GSM309125': [0], 'GSM309126': [0], 'GS

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1003', '1005', '1006', '1007', '1009', '1010', '1011', '1012', '1014',
       '1018', '1020', '1021', '1027', '1032', '1034', '1035', '1038', '1040',
       '1046', '1051'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1001', '1002', '1003', '1004', '1005'], 'ProbeID': ['1405_i_at', '1773_at', '200024_at', '200033_at', '200063_s_at'], 'Length': ['69', '69', '70', '70', '70'], 'TM': [76.668, 81.422, 78.843, 74.743, 74.743], 'Conc. (uM)': [50.0, 50.0, 50.0, 50.0, 50.0], 'Vol. (uL)': [200.0, 200.0, 200.0, 200.0, 200.0], 'SEQUENCE': ['AAAAGCTTCCCCAACTAAAGCCTAGAAGAGCTTCTGAGGCGCTGCTTTGTCAAAAGGAAGTCTCTAGGT', 'AGCTTAAGGATGAGACATCGGCAGAGCCTGCAACCGACTAGAGGACCTGGGTCCCGGCAGCTCTTTGCT', 'TCCTCGAACTCCTATGCCATTAAGAAGAAGGACGAGCTGGAGCGTGTGGCCAAGTCCAACCGCTGATTTT', 'TGCTACTGCAGCTGCACCTATGATTGGTTATCCAATGCCAACAGGATATTCCCAATAAGACTTTAGAAGT', 'AGAGTGAGAACTTTCCCTACCGTGTTTGATAAATGTTGTCCAGGTTCTATTGCCAAGAATGTGTTGTCCA'], 'Gene Symbol': ['CCL5', 'FNTB', '---', 'DDX5', 'NPM1'], 'ORF': ['CCL5', 'FNTB', nan, 'DDX5', 'NPM1'], 'SPOT_ID': [nan, nan, '200024_at', nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the corresponding key names from the gene annotation preview
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function
gene_mapping = get_gene_mapping(gene_annotation, prob_col=identifier_key, gene_col=gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Essential_Thrombocythemia/gene_data/GSE12295.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Essential_Thrombocythemia')

# 4. Save the cohort information.
save_cohort_info('GSE12295', './preprocessed/Essential_Thrombocythemia/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Essential_Thrombocythemia/GSE12295.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Essential_Thrombocythemia', the least common label is '1.0' with 24 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Essential_Thrombocythemia' in this dataset is fine.

